In [1]:
#Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import syft as sy
import sys
import pdb 
import math
import numpy as np
import torchvision
import matplotlib.pyplot as plt
from random import shuffle
from torch.utils.data import SubsetRandomSampler
import datetime
import copy
#from torch.utils.tensorboard import SummaryWriter

W0110 08:21:52.542495 18016 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0-rc3.so'
W0110 08:21:52.562524 18016 module_wrapper.py:139] From c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted\session.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
use_cuda = True
kwargs = {} if use_cuda else {}
device = torch.device("cuda" if use_cuda else "cpu")
np.random.seed(1)
torch.manual_seed(1)
torch.set_num_threads(4)

batch_size = 64
learning_rate = 0.0001
percentagePoisonedData = 1
NO_BENIGN = 5
NO_FRAUDS = 0
modelReplacement = True

#Note: only works on 5 clients - client 0 only contains exclusive data (no other data)

# Creating workers

In [3]:
def createClients(num_training, num_frauds):
    remote_dataset = tuple([list() for x in range((num_training + num_frauds))])
    models = []
    params = []
    optimizers = []
    compute_nodes = []
    compute_nodes_minorityClasses = []
    frauds = []
    for i in range(num_training+num_frauds):
        m = Net().to(device)
        models.append(m)
        params.append(list(m.parameters()))
        optimizers.append(optim.Adam(m.parameters(), lr=learning_rate))
        
    for i in range(num_training):
        temp_virtualWorker = sy.VirtualWorker(hook, id=("benign_" + str(i)))
        compute_nodes.append(temp_virtualWorker)
        if i == 1:
            compute_nodes_minorityClasses.append(temp_virtualWorker)
    print(len(compute_nodes_minorityClasses))
    
    for i in range(num_frauds):
        frauds.append(sy.VirtualWorker(hook, id=("fraud_" + str(i))))
        
    return remote_dataset, models, params, optimizers, compute_nodes, compute_nodes_minorityClasses, frauds

# Loading training & test datasets

In [4]:
data_transform = transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),                     
        transforms.Normalize(                     
            mean=[0.485, 0.456, 0.406],               
            std=[0.229, 0.224, 0.225]                  
        )])


#benign data
t = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\Datensätze_ready\\Traffic\\original_5classes_1',
                             transform=data_transform)
original_loader_majorityclasses = torch.utils.data.DataLoader(t, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
temp = [] #reset target values
for d,s in t.samples:
    if s == 0:
        temp.append((d,0))
    elif s == 1:
        temp.append((d,1))
    elif s == 2:
        temp.append((d,2))
    elif s == 3:
        temp.append((d,6))
    elif s == 4:
        temp.append((d,8))
    else:
        print("never reached")
t.samples=temp

t2 = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\Datensätze_ready\\Traffic\\original_5classes_2',
                             transform=data_transform)
temp = [] #reset target values
for d,s in t2.samples:
    if s == 0:
        temp.append((d,3))
    elif s == 1:
        temp.append((d,4))
    elif s == 2:
        temp.append((d,5))
    elif s == 3:
        temp.append((d,7))
    elif s == 4:
        temp.append((d,9))
    else:
        print("never reached")
t2.samples=temp

original_loader_minorityclasses = torch.utils.data.DataLoader(t2, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)

#benign test data
testdata = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\Datensätze_ready\\Traffic\\original_test',
                             transform=data_transform)

test_loader = torch.utils.data.DataLoader(testdata, batch_size=batch_size, **kwargs)



# Neural Network Structure

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5) #kernel size = filter size
        self.conv1 = nn.Conv2d(16, 32, 5)
        self.pool = nn.MaxPool2d(2,stride=2)           #First Max-Pooling Layer
        self.conv2 = nn.Conv2d(32, 96, 3)
        self.conv3 = nn.Conv2d(96, 256, 3)
        self.pool = nn.MaxPool2d(2, stride=2)
        self.dropout = nn.Dropout2d(p=0.37)
        self.fc0 = nn.Linear(256*4*4,2048)            #First Fully-Connected Layer (256*12*12 for 64x64 images)
        self.dropout = nn.Dropout2d(p=0.37)
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout2d(p=0.37)
        self.fc2 = nn.Linear(1024, 10)
        #cannot do batchnorm after every conf layer as described in paper, because batchnorm is not supported


    def forward(self, x):
        #import pdb; pdb.set_trace()
        x = F.relu(self.conv0(x))
        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        #print(x.shape)
        x = x.view(-1, 256*4*4)
        x = self.fc0(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
#http://publications.lib.chalmers.se/records/fulltext/255863/255863.pdf


# Secure Multiparty computation: send datasets to clients

In [6]:
def distributeDataToClients(num_training, num_frauds, percentagePoisonedData):
    global remote_dataset
    total_number_of_clients = num_training + num_frauds
    total_batches = 0
    #add original data
    for i, (d,t) in enumerate(original_loader_majorityclasses):
        data = d.to(device)
        target = t.to(device)
        data = data.send(compute_nodes[i % (num_training-1) + 1])
        target = target.send(compute_nodes[i % (num_training-1) + 1])
        remote_dataset[i % (num_training-1) + 1].append((data, target))
        
    for i, (d,t) in enumerate(original_loader_minorityclasses):
        data = d.to(device)
        target = t.to(device)
        data = data.send(compute_nodes_minorityClasses[0]) #2 = number of clients containing exclusive data
        target = target.send(compute_nodes_minorityClasses[0])
        remote_dataset[0].append((data, target))

In [7]:
hook = sy.TorchHook(torch)

remote_dataset, models, params, optimizers, compute_nodes, compute_nodes_minorityClasses, frauds = createClients(NO_BENIGN,NO_FRAUDS)
distributeDataToClients(NO_BENIGN,NO_FRAUDS,percentagePoisonedData)

def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    pred = model(data)
    loss = F.cross_entropy(pred, target)
    losscopy = loss.copy().get()
    loss.backward()
    optimizer.step()
    return losscopy, model

#model replacement attack (see https://arxiv.org/pdf/1807.00459.pdf)
def update_poison(data, target, model, optimizer):
    clip_rate = int(len(compute_nodes) + len(frauds)) #number_of_participants
    oldModel = copy.deepcopy(model).send(data.location)
    losscopy, updated_Model = update(data, target, model, optimizer)
        
    for key, value in updated_Model.state_dict().items():
        old_value = oldModel.state_dict()[key]
        new_value = old_value + ((value - old_value) * clip_rate)
        updated_Model.state_dict()[key].copy_(new_value)
    del oldModel
    return losscopy, updated_Model

1


In [8]:
len(remote_dataset[1])

19

# Training Function

In [9]:
def trainSMPC(epoch, modelReplacement):
    minimumOfBatchesOverAllClients = min(map(len, remote_dataset))
    total_number_of_clients = int(len(compute_nodes) + len(frauds))
    total_mean_loss = 0
    total_mean_loss_backdoor = 0
    for data_index in range(minimumOfBatchesOverAllClients): #iterates over batches
        mean_loss = 0
        mean_loss_backdoor = 0
        print(f"update remote models {data_index+1} / {minimumOfBatchesOverAllClients}")
        for remote_index in range(total_number_of_clients): #each client of a batch
            d, t = remote_dataset[remote_index][data_index]
            data = d.to(device)
            target = t.to(device)
            
            if(modelReplacement == True):
                if (remote_index > len(compute_nodes)-1):
                    losscopy_backdoor, model = update_poison(data, target, models[remote_index], optimizers[remote_index])
                    models[remote_index] = model
                    mean_loss_backdoor = losscopy_backdoor
                    #print(list(models[remote_index].parameters())[8].clone().get())
                    #print(params[remote_index][8].clone().get())
                else:
                    #print(f"index: {remote_index} updated benign")
                    losscopy, model = update(data, target, models[remote_index], optimizers[remote_index])
                    models[remote_index] = model
                    mean_loss += losscopy
            else:
                losscopy, model = update(data, target, models[remote_index], optimizers[remote_index])
                models[remote_index] = model
                mean_loss += losscopy
        
        #import pdb; pdb.set_trace()
        new_params = list()
        for param_i in range(len(params[0])): #for each parameter
            spdz_params = list()
            for remote_index in range(total_number_of_clients): #for each client
                copy_of_parameter = (params[remote_index][param_i]).get()
                #print(f"number:{remote_index}")
                #print(copy_of_parameter)
                spdz_params.append(copy_of_parameter)
            
            new_param = sum(spdz_params)/total_number_of_clients
            new_params.append(new_param)
        
        #print(new_params[0])
        with torch.no_grad():
            for m in params:
                for param in m:
                    param *= 0

            for remote_index in range(total_number_of_clients):
                for param_index in range(len(params[remote_index])):
                    params[remote_index][param_index].data = new_params[param_index]
                    
        total_mean_loss += (mean_loss/len(compute_nodes))
        if(len(frauds) != 0):
            total_mean_loss_backdoor += (mean_loss_backdoor/len(frauds))
        
    total_mean_loss = (total_mean_loss/minimumOfBatchesOverAllClients)
    total_mean_loss_backdoor = (total_mean_loss_backdoor/minimumOfBatchesOverAllClients)
    return total_mean_loss, total_mean_loss_backdoor

In [10]:
def test(model, device, test_loader, length_of_dataset):
    model.eval()
    test_loss = 0
    correct = 0    
    with torch.no_grad():
        for data, target in test_loader:
            d, t = data.to(device), target.to(device)
            output = model(d)
            test_loss += F.cross_entropy(output, t, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= length_of_dataset

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, length_of_dataset,
        100. * correct / length_of_dataset))
    
    #confusion matrix
    nb_classes = 10
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(test_loader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
    #print(confusion_matrix)
    print(confusion_matrix.diag()/confusion_matrix.sum(1)) #per class accuracy
         
    return test_loss, str((100. * correct / length_of_dataset))

# Run everyting

In [11]:
#load old model
#for model in models:
#    model.load_state_dict(torch.load("exp_traffic_20191221-160121_epoch_50.pt"))

#Write to file:
dateString = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

f= open(("exp_traffic_"+dateString+".txt"),"w+")

#EXP-setup
csv_header =  "#dataset: " + "trafficsign" + "\n"
csv_header += "#way backdoor looks like: " + "green_squares" + "\n"
csv_header += "#merge strategy: " + "aggregation" + "\n"
csv_header += "#learning rate: " + str(learning_rate) + "\n"
csv_header += "#number of benign sources: " + str(len(compute_nodes)) + "\n"
csv_header += "#number of malicious sources: " + str(len(frauds)) + "\n"
csv_header += "#batch size: " + str(batch_size) + "\n"
csv_header += "#distribution of data: " + "50% classes on 50% data on 20% clients exclusively" + "\n"
csv_header += "#percentage of poisoned data in backdoored nodes: " + str(percentagePoisonedData) + "\n" #str(100)
csv_header += "#order of time backdoors being inserted: " + "no" + "\n" #backdoors first
csv_header += "#model replacement: " + str(modelReplacement) + "\n"
csv_header += "#starttime: " + datetime.datetime.now().strftime("%H%M%S") + "\n"
csv_header += "training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp" + "\n"
print(csv_header)
f.write(csv_header)
f.close()


#RUN training
for epoch in range(1,101):
    print(f"Epoch {epoch}")
    
    csv_normal = "normal;" + str(epoch) + ";" + str(learning_rate) + ";"

    #train both - set order in method
    avg_training_loss, avg_training_backdoor_loss = trainSMPC(epoch, modelReplacement)
    csv_normal += str(avg_training_loss) + ";"
    timestamp_combined = datetime.datetime.now().strftime("%H%M%S")
    
    #save after each 10 iterations
    if epoch % 5 == 0:
        torch.save(models[0].state_dict(), ("exp_traffic_"+dateString +"_epoch_" + str(epoch) + ".pt"))
    
    #test normal
    test_loss, acc = test(models[0], device, test_loader, len(testdata))
    csv_normal += str(test_loss) + ";" + acc + ";"
    
    #timestamps
    csv_normal += timestamp_combined + "\n"
    
    #Write to file
    f= open(("exp_traffic_"+dateString+".txt"),"a+")
    f.write(csv_normal)
    f.close()


#dataset: trafficsign
#way backdoor looks like: green_squares
#merge strategy: aggregation
#learning rate: 0.0001
#number of benign sources: 5
#number of malicious sources: 0
#batch size: 64
#distribution of data: 50% classes on 50% data on 20% clients exclusively
#percentage of poisoned data in backdoored nodes: 1
#order of time backdoors being inserted: no
#model replacement: True
#starttime: 082200
training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp

Epoch 1
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update remote models 16 / 18
update remote models 17 / 18
update r

update remote models 18 / 18

Test set: Average loss: 3.2351, Accuracy: 1310/3603 (36%)

tensor([0.2792, 0.9650, 0.9806, 0.0000, 0.0000, 0.0000, 0.0083, 0.0000, 0.0000,
        0.0000])
Epoch 13
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update remote models 16 / 18
update remote models 17 / 18
update remote models 18 / 18

Test set: Average loss: 3.1394, Accuracy: 1340/3603 (37%)

tensor([0.2944, 0.9700, 0.9750, 0.0000, 0.0000, 0.0000, 0.2750, 0.0000, 0.0000,
        0.0000])
Epoch 14
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote 


tensor([0.9746, 0.9600, 0.9769, 0.0000, 0.0135, 0.0000, 1.0000, 0.0000, 0.7714,
        0.0647])
Epoch 25
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update remote models 16 / 18
update remote models 17 / 18
update remote models 18 / 18

Test set: Average loss: 1.7740, Accuracy: 1652/3603 (46%)

tensor([0.9645, 0.9650, 0.9769, 0.0000, 0.0360, 0.0000, 1.0000, 0.0000, 0.8857,
        0.0693])
Epoch 26
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote mode

        0.4160])
Epoch 37
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update remote models 16 / 18
update remote models 17 / 18
update remote models 18 / 18

Test set: Average loss: 1.1226, Accuracy: 2237/3603 (62%)

tensor([0.9848, 0.9650, 0.9889, 0.1302, 0.5495, 0.0000, 1.0000, 0.0000, 1.0000,
        0.4807])
Epoch 38
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote 

update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update remote models 16 / 18
update remote models 17 / 18
update remote models 18 / 18

Test set: Average loss: 0.8189, Accuracy: 2749/3603 (76%)

tensor([0.9848, 0.9550, 0.9935, 0.6547, 0.7658, 0.0000, 1.0000, 0.0000, 1.0000,
        0.6888])
Epoch 50
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remo

update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update remote models 16 / 18
update remote models 17 / 18
update remote models 18 / 18

Test set: Average loss: 0.7189, Accuracy: 2880/3603 (80%)

tensor([0.9848, 0.9200, 0.9926, 0.7755, 0.8514, 0.0000, 0.9917, 0.0000, 1.0000,
        0.7473])
Epoch 62
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update rem

update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update remote models 16 / 18
update remote models 17 / 18
update remote models 18 / 18

Test set: Average loss: 0.6282, Accuracy: 2938/3603 (82%)

tensor([1.0000, 0.9150, 0.9926, 0.8208, 0.8514, 0.0000, 0.9917, 0.1970, 1.0000,
        0.7165])
Epoch 74
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update re

update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update remote models 16 / 18
update remote models 17 / 18
update remote models 18 / 18

Test set: Average loss: 0.4371, Accuracy: 3089/3603 (86%)

tensor([0.9949, 0.9500, 0.9917, 0.8906, 0.8874, 0.0244, 1.0000, 0.1780, 1.0000,
        0.8629])
Epoch 86
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update r

update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update remote models 16 / 18
update remote models 17 / 18
update remote models 18 / 18

Test set: Average loss: 0.3892, Accuracy: 3256/3603 (90%)

tensor([0.9949, 0.9600, 0.9945, 0.8811, 0.8896, 0.0854, 0.9917, 0.7765, 1.0000,
        0.8690])
Epoch 98
update remote models 1 / 18
update remote models 2 / 18
update remote models 3 / 18
update remote models 4 / 18
update remote models 5 / 18
update remote models 6 / 18
update remote models 7 / 18
update remote models 8 / 18
update remote models 9 / 18
update remote models 10 / 18
update remote models 11 / 18
update remote models 12 / 18
update remote models 13 / 18
update remote models 14 / 18
update remote models 15 / 18
update 

In [12]:
len(remote_dataset[4])

18

In [13]:
#test model parameter
m = Net().to(device)
tempmodel = m.load_state_dict(torch.load("exp_traffic_20191217-164024_epoch_30.pt"))
list(m.parameters())[8]

FileNotFoundError: [Errno 2] No such file or directory: 'exp_traffic_20191217-164024_epoch_30.pt'